In [3]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
import hvplot.pandas
import requests
import geopandas as gpd
import fiona as fn

In [4]:
# Importing Medicare data
medicare_data_df = pd.read_csv("Data\Medicare_Telehealth_Trends_Q1_2023.csv")
medicare_data_df.head()


,Year,quarter,Bene_Geo_Desc,Bene_Mdcd_Mdcr_Enrl_Stus,Bene_Race_Desc,Bene_Sex_Desc,Bene_Mdcr_Entlmt_Stus,Bene_Age_Desc,Bene_RUCA_Desc,Total_Bene_TH_Elig,Total_PartB_Enrl,Total_Bene_Telehealth,Pct_Telehealth
0,2023,1,Wyoming,Medicare Only,All,All,All,All,All,57582.0,87532.0,3976.0,0.0690
1,2023,1,Wyoming,Medicare & Medicaid,All,All,All,All,All,7781.0,10302.0,1373.0,0.1765
2,2023,1,Wyoming,All,Other/Unknown,All,All,All,All,NaN,NaN,NaN,NaN
3,2023,1,Wyoming,All,Non-Hispanic White,All,All,All,All,59795.0,89194.0,4813.0,0.0805
4,2023,1,Wyoming,All,Hispanic,All,All,All,All,2476.0,3786.0,213.0,0.0860


In [5]:
#Checking where NaN values come from
medicare_data_df.loc[medicare_data_df["Total_Bene_TH_Elig"].isna()].groupby(["Bene_Race_Desc","Bene_RUCA_Desc"]).count()

Year  quarter  Bene_Geo_Desc  \
Bene_Race_Desc                Bene_RUCA_Desc                                 
All                           All               16       16             16   
                              Rural             16       16             16   
                              Unknown          864      864            864   
American Indian/Alaska Native All                8        8              8   
                              Unknown           16       16             16   
Asian/Pacific Islander        Unknown           16       16             16   
Black/African American        Unknown           16       16             16   
Hispanic                      Unknown           16       16             16   
Non-Hispanic White            Unknown           16       16             16   
Other/Unknown                 All             1008     1008           1008   
                              Rural             16       16             16   
                              Unknown           16       16             16   
                              Urban             16       16             16   

                                              Bene_Mdcd_Mdcr_Enrl_Stus  \
Bene_Race_Desc                Bene_RUCA_Desc                             
All                           All                                   16   
                              Rural                                 16   
                              Unknown                              864   
American Indian/Alaska Native All                                    8   
                              Unknown                               16   
Asian/Pacific Islander        Unknown                               16   
Black/African American        Unknown                               16   
Hispanic                      Unknown                               16   
Non-Hispanic White            Unknown                               16   
Other/Unknown                 All                                 1008   
                              Rural                                 16   
                              Unknown                               16   
                              Urban                                 16   

                                              Bene_Sex_Desc  \
Bene_Race_Desc                Bene_RUCA_Desc                  
All                           All                        16   
                              Rural                      16   
                              Unknown                   864   
American Indian/Alaska Native All                         8   
                              Unknown                    16   
Asian/Pacific Islander        Unknown                    16   
Black/African American        Unknown                    16   
Hispanic                      Unknown                    16   
Non-Hispanic White            Unknown                    16   
Other/Unknown                 All                      1008   
                              Rural                      16   
                              Unknown                    16   
                              Urban                      16   

                                              Bene_Mdcr_Entlmt_Stus  \
Bene_Race_Desc                Bene_RUCA_Desc                          
All                           All                                16   
                              Rural                              16   
                              Unknown                           864   
American Indian/Alaska Native All                                 8   
                              Unknown                            16   
Asian/Pacific Islander        Unknown                            16   
Black/African American        Unknown                            16   
Hispanic                      Unknown                            16   
Non-Hispanic White            Unknown                            16   
Other/Unknown                 All    

In [6]:
# Cleaning up columns
cleaned_data = medicare_data_df.rename(columns={"Bene_Geo_Desc": "State", "Bene_Mdcd_Mdcr_Enrl_Stus": "Medicare/Medicaid","Bene_Race_Desc": "Race",
                                 "Bene_Sex_Desc": "Sex", "Bene_Mdcr_Entlmt_Stus":"Entitlement Status", "Bene_Age_Desc":"Age","Bene_RUCA_Desc":"Locality","Total_Bene_TH_Elig":"Total Telehealth-Eligible Users",
                                 "Total_PartB_Enrl":"Part B Enrollment","Total_Bene_Telehealth":"Telehealth Users","Pct_Telehealth":"Percent of Telehealth Users"}).dropna().reset_index(drop=True)
cleaned_data.head()

,Year,quarter,State,Medicare/Medicaid,Race,Sex,Entitlement Status,Age,Locality,Total Telehealth-Eligible Users,Part B Enrollment,Telehealth Users,Percent of Telehealth Users
0,2023,1,Wyoming,Medicare Only,All,All,All,All,All,57582.0,87532.0,3976.0,0.0690
1,2023,1,Wyoming,Medicare & Medicaid,All,All,All,All,All,7781.0,10302.0,1373.0,0.1765
2,2023,1,Wyoming,All,Non-Hispanic White,All,All,All,All,59795.0,89194.0,4813.0,0.0805
3,2023,1,Wyoming,All,Hispanic,All,All,All,All,2476.0,3786.0,213.0,0.0860
4,2023,1,Wyoming,All,Black/African American,All,All,All,All,263.0,462.0,33.0,0.1255


In [7]:
#Checking total counts by year
cleaned_data.groupby("Year").count()

,quarter,State,Medicare/Medicaid,Race,Sex,Entitlement Status,Age,Locality,Total Telehealth-Eligible Users,Part B Enrollment,Telehealth Users,Percent of Telehealth Users
Year,,,,,,,,,,,,
2020,5374,5374,5374,5374,5374,5374,5374,5374,5374,5374,5374,5374
2021,5380,5380,5380,5380,5380,5380,5380,5380,5380,5380,5380,5380
2022,5375,5375,5375,5375,5375,5375,5375,5375,5375,5375,5375,5375
2023,1074,1074,1074,1074,1074,1074,1074,1074,1074,1074,1074,1074


In [9]:
#Separating National data from State data
national = cleaned_data.loc[(cleaned_data["State"] == "National") & (cleaned_data['quarter']!="Overall")]
state = cleaned_data.loc[(cleaned_data["State"] != "National") & (cleaned_data['quarter']!="Overall")]

In [10]:
overall_states = state.loc[(state["Age"] == "All") & (state['Entitlement Status']== "All") & (state['Locality']== "All") & (state["Medicare/Medicaid"]== "All") & (state['Race']== "All")
                            & (state['Sex']== "All")].reset_index(drop=True).groupby(["State"])["Percent of Telehealth Users"]
states_means = overall_states.mean()
states_min = overall_states.min()
states_max = overall_states.max()
states_range = states_max - states_min

In [11]:
states_polygons = open("Data/wkt_all_states.txt",'r').read().split('\n')
state_names = []
state_gons = []
for x in range(0,len(states_polygons)):
    if x % 3 == 1:
        state_names.append(states_polygons[x])
    if x % 3 == 2:
        state_gons.append(states_polygons[x])
range_df = pd.DataFrame(states_range).reset_index()
polygons_df = pd.DataFrame(state_gons,state_names).reset_index().rename(columns={"index":"State"}).sort_values(by= "State")
range_matched = range_df.loc[(range_df["State"]!="District Of Columbia") & (range_df["State"]!="Puerto Rico")& (range_df["State"]!="Virgin Islands")& (range_df["State"]!="Territories")].sort_values(by="State").reset_index(drop=True)
range_matched.drop(["State"],axis=1,inplace=True)
combined_states = pd.concat([range_matched,polygons_df],join='outer',axis=1)
combined_states.rename(columns={0:"geometry"},inplace=True)
combined_states

,Percent of Telehealth Users,State,geometry
0,0.3244,Alaska,"POLYGON((-141.0205 70.0187,-141.7291 70.1292,-..."
1,0.3808,Alabama,"POLYGON((-88.1955 35.0041,-85.6068 34.9918,-85..."
2,0.3867,Arkansas,"POLYGON((-94.0416 33.0225,-91.2057 33.0075,-91..."
3,0.3157,Arizona,"POLYGON((-112.5989 36.9993,-110.8630 37.0004,-..."
4,0.4610,California,"POLYGON((-124.4009 41.9983,-123.6237 42.0024,-..."
5,0.3637,Colorado,"POLYGON((-109.0448 37.0004,-102.0424 36.9949,-..."
6,0.5483,Connecticut,"POLYGON((-73.4875 42.0498,-73.4247 42.0511,-72..."
7,0.5376,Delaware,"POLYGON((-75.7919 39.7188,-75.7837 39.5210,-75..."
8,0.3754,Florida,"POLYGON((-87.6050 30.9988,-86.5613 30.9964,-85..."
9,0.3412,Georgia,"POLYGON((-85.6082 34.9974,-84.7266 34.9906,-84..."


In [12]:
combined_states['geometry'] = gpd.GeoSeries.from_wkt(combined_states['geometry'])

In [14]:
my_geo_df = gpd.GeoDataFrame(combined_states, geometry='geometry')
my_geo_df.hvplot(color="Percent of Telehealth Users",height=400,width=600,tiles="OSM")

:Overlay
   .Tiles.I    :Tiles   [x,y]
   .Polygons.I :Polygons   [Longitude,Latitude]   (Percent of Telehealth Users)

In [21]:
gpd.read_file("data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp")

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
0,28,01779790,0400000US28,28,MS,Mississippi,00,121533519481,3926919758,"MULTIPOLYGON (((-88.50297 30.21523, -88.49176 ..."
1,37,01027616,0400000US37,37,NC,North Carolina,00,125923656064,13466071395,"MULTIPOLYGON (((-75.72681 35.93584, -75.71827 ..."
2,40,01102857,0400000US40,40,OK,Oklahoma,00,177662925723,3374587997,"POLYGON ((-103.00257 36.52659, -103.00219 36.6..."
3,51,01779803,0400000US51,51,VA,Virginia,00,102257717110,8528531774,"MULTIPOLYGON (((-75.74241 37.80835, -75.74151 ..."
4,54,01779805,0400000US54,54,WV,West Virginia,00,62266474513,489028543,"POLYGON ((-82.64320 38.16909, -82.64300 38.169..."
5,22,01629543,0400000US22,22,LA,Louisiana,00,111897594374,23753621895,"MULTIPOLYGON (((-88.86770 29.86155, -88.86566 ..."
6,26,01779789,0400000US26,26,MI,Michigan,00,146600952990,103885855702,"MULTIPOLYGON (((-83.19159 42.03537, -83.18993 ..."
7,25,00606926,0400000US25,25,MA,Massachusetts,00,20205125364,7129925486,"MULTIPOLYGON (((-70.23405 41.28565, -70.22361 ..."
8,16,01779783,0400000US16,16,ID,Idaho,00,214049787659,2391722557,"POLYGON ((-117.24267 44.39655, -117.23484 44.3..."
9,12,00294478,0400000US12,12,FL,Florida,00,138949136250,31361101223,"MULTIPOLYGON (((-80.17628 25.52505, -80.17395 ..."
